In [1]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm, notebook

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [2]:
consumer_api_key = '2Em7SxlX9jPMfL4x97r3zMO0x'
consumer_api_secret = "sVbJzekKuiAgq83Y7gCwNVbSowqQokGVzWexKHl2cXIPceWtSd"

In [3]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [4]:
api = tw.API(auth, wait_on_rate_limit=True)

In [5]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(1245)

In [6]:
tweets_copy = []
for tweet in tqdm(tweets):
     tweets_copy.append(tweet)

1245it [00:44, 27.67it/s]


In [7]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 1245


In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 1245/1245 [10:14<00:00,  2.03it/s]


In [10]:
tweets_df.head()


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,COVID Scale,Earth,A bot designed to put the #COVID19 pandemic's effect on the United States into historical perspective. Stats from @Worldometers,2017-09-15 19:54:39,1371,3,7,False,2022-11-27 18:30:59,"Since the start of the pandemic, 1,104,751 Americans have died from #COVID19 (16.6% of all deaths worldwide). That is about the same as 1,203 Jonestowns which killed 918 people in 1978: https://t.co/cAXtZeSSjF",[COVID19],HN_Comments,False
0,Olivia,,"Cleveland rocks. The Beatles, Neil Young , Bob Dylan and CCR rock my world. “Our lives begin to end the day we become silent about things that matter.” MLKJr",2011-06-09 22:09:27,789,1864,25960,False,2022-11-27 18:30:07,It’s not over folks. #GetVaccinated #COVID19 #CovidIsNotOver #cov https://t.co/n7H6Woe0GK,"[GetVaccinated, COVID19, CovidIsNotOver, cov]",Twitter for iPhone,False
0,Truetelehealth,,"TrueTeleHealth is a comprehensive, easy to use, and affordable telehealth platform. It is created to improve access and delivery of virtual healthcare.",2020-03-30 08:34:10,771,1100,41,False,2022-11-27 18:30:00,How TeleHealth can enhance your practice revenue\n\nhttps://t.co/L1PgvyQ16o\n\n#telehealth #telemedicine #healthcare #therapy #primarycare #home #HIT #covid19 #usa #selfcare #coronavirus #photooftheday https://t.co/n2UvQSLgGZ,"[telehealth, telemedicine, healthcare]",Twitter Web App,False
0,Wyldcliffe,no fixed address,Ponderer & Wanderer,2022-10-05 03:39:36,6,9,687,False,2022-11-27 18:29:56,#blankpaperexercise\nThe Chinese people need our help! Trend blank paper exercise get the word out that the Chinese people in spite of horrific consequences are standing up against these lockdowns. The world needs a blank paper exercise. Time for courage is now. #China #COVID19 https://t.co/zUCh2YmkRy,[blankpaperexercise],Twitter for iPad,False
0,The Swiss Pharmacy,"Geneva, Switzerland",Leading supplier of branded and generic medicines online,2009-08-02 10:46:42,142,2,38,False,2022-11-27 18:29:42,How long does covid-19 survive on surfaces ? The effect of temperature on persistence of SARS-CoV-2 on common surfaces #covid19 #covidsurfacetransmission https://t.co/S8WLjHDBeB,None,Twitter Web App,False


In [13]:
tweets_old_df = pd.read_csv("covid19tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")


past tweets: (82710, 13)


In [14]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 1245 past tweets: 82710 all tweets: 83955


In [15]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")


all tweets: (83953, 13)


In [16]:
tweets_all_df.to_csv("covid19tweets.csv", index=False)